In [5]:
# !pip install geopandas folium matplotlib seaborn scipy
# !pip install esda
# !pip install splot
# !pip install geopandas contextily
# # for google colab, had to reinstall some pacakges.

In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import datetime as dt
import scipy

from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from shapely.geometry import Point
from sklearn.neighbors import NearestNeighbors

# visualization
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
import seaborn as sns
import folium
from folium.plugins import HeatMap
from folium import Marker
from folium.plugins import MarkerCluster
import plotly.express as px
import plotly.io as pio


import contextily as ctx
from scipy.stats import f_oneway
from sklearn.decomposition import PCA
from functools import reduce
from scipy.stats import chi2_contingency
import statsmodels.api as sm

# spatial statistics
from esda.moran import Moran
from esda.getisord import G_Local
from libpysal.weights import Queen, Rook

# system and utility
import warnings
import os
import io
from IPython.display import IFrame
from google.colab import files

from libpysal.weights import Queen, Rook
from esda.moran import Moran
import matplotlib.pyplot as plt
from splot.esda import moran_scatterplot

# suppress warnings
warnings.filterwarnings('ignore')

# inline
%matplotlib inline

In [6]:
pd.set_option('display.float_format', lambda x: '%.4f' % x)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# **Step 1 Get the Rent data**


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
# data source:
link1 = '/content/drive/My Drive/X999/rent_median_df_all.csv'
link2 = '/content/drive/My Drive/X999/rent_median_df_br.csv'
link3 = '/content/drive/My Drive/X999/rent_median_df_bk.csv'
link4 = '/content/drive/My Drive/X999/rent_median_df_man.csv'
link5 = '/content/drive/My Drive/X999/rent_median_df_queens.csv'
link6 = '/content/drive/My Drive/X999/rent_median_df_si.csv'

# eviction related
link7 = '/content/drive/My Drive/X999/bbl_evictions_311_svi_normal_times_correct_coordinates.csv'
link8 = '/content/drive/My Drive/X999/bbl_evictions_311_svi_covid_correct_coordinates.csv'

In [12]:
evictions_df = pd.read_csv(link7)
evictions_df_covid = pd.read_csv(link8)

In [184]:
all_rent = pd.read_csv(link1)
br_rent = pd.read_csv(link2)
bk_rent = pd.read_csv(link3)
man_rent = pd.read_csv(link4)
queens_rent = pd.read_csv(link5)
si_rent = pd.read_csv(link6)

In [17]:
type(evictions_df.executed_date[0])

str

In [18]:
evictions_df.executed_date = pd.to_datetime(evictions_df.executed_date)
evictions_df_covid.executed_date = pd.to_datetime(evictions_df_covid.executed_date)

In [21]:
evictions_df.columns

Index(['primary_key', 'bbl', 'court_index_number', 'docket_number',
       'eviction_address', 'eviction_apartment_number', 'executed_date',
       'borough', 'zipcode', 'ejectment', 'eviction/legal_possession',
       'latitude', 'longitude', 'community_board', 'council_district',
       'census_tract', 'bin', 'nta', 'year', 'month_year', 'geometry',
       'average_year_eviction_count', 'yearbuilt', 'bldgclass', 'numfloors',
       'unitsres', 'ownername', 'bldgarea', 'building_type',
       'building_category', 'is_condo', 'floor_category', 'rent_era',
       'architectural_style', 'economic_period', 'residential_units_category',
       'is_llc', 'building_size_category', 'size_quartile', 'decade', 'fips',
       'e_totpop', 'rpl_theme1', 'rpl_theme2', 'rpl_theme3', 'rpl_theme4',
       'rpl_themes', 'ep_pov150', 'ep_unemp', 'ep_nohsdp', 'ep_uninsur',
       'ep_age65', 'ep_age17', 'ep_disabl', 'ep_limeng', 'ep_noveh',
       'ep_crowd', 'ep_hburd', 'ep_afam', 'ep_hisp', 'ep_asian',

In [19]:
evictions_time_series = evictions_df.groupby('executed_date').count().reset_index()
evictions_time_series_covid = evictions_df_covid.groupby('executed_date').count().reset_index()

In [20]:
evictions_time_series.head()

,executed_date,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,average_year_eviction_count,yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,svi_group,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,average_year_eviction_unit_count,average_year_eviction_nta_count,evictions_nta_per_1k,evictions_nta_per_unit_per_1k,evictions_nta_per_1k.1
0,2017-01-03,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77
1,2017-01-04,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
2,2017-01-05,82,82,82,82,82,82,82,82,82,82,81,81,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82
3,2017-01-06,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105
4,2017-01-09,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77


In [22]:
manhattan_evictions = evictions_df[evictions_df['borough'] == 'MANHATTAN']
brooklyn_evictions = evictions_df[evictions_df['borough'] == 'BROOKLYN']
queens_evictions = evictions_df[evictions_df['borough'] == 'QUEENS']
bronx_evictions = evictions_df[evictions_df['borough'] == 'BRONX']
staten_evictions = evictions_df[evictions_df['borough'] == 'STATEN ISLAND']

In [23]:
manhattan_evictions_covid = evictions_df_covid[evictions_df_covid['borough'] == 'MANHATTAN']
brooklyn_evictions_covid = evictions_df_covid[evictions_df_covid['borough'] == 'BROOKLYN']
queens_evictions_covid = evictions_df_covid[evictions_df_covid['borough'] == 'QUEENS']
bronx_evictions_covid = evictions_df_covid[evictions_df_covid['borough'] == 'BRONX']
staten_evictions_covid = evictions_df_covid[evictions_df_covid['borough'] == 'STATEN ISLAND']

# **Step 2: Seperate the eviction timeseries by borough and one for all boroughs**

## **Step 2.1: Normal times**

In [56]:
manhattan_evictions_copy = manhattan_evictions.copy()
# make a copy, otherwise, a warning
manhattan_evictions_copy['month'] = manhattan_evictions_copy['executed_date'].dt.to_period('M')
manhattan_evictions_monthly = manhattan_evictions_copy.groupby('month').count()['primary_key']
man_monthly_df = manhattan_evictions_monthly.reset_index()
man_monthly_df.columns = ['month', 'eviction_count']
man_monthly_df.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59
month,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
eviction_count,254,183,215,211,217,211,206,248,202,242,173,110,222,173,227,225,234,203,201,208,160,179,154,136,208,182,166,223,169,181,146,155,110,160,121,114,126,117,123,120,114,207,172,211,184,201,194,113,235,211,158,187,158,165,170,202,175,291,168,167


In [57]:
bronx_evictions_copy = bronx_evictions.copy()
bronx_evictions_copy['month'] = bronx_evictions_copy['executed_date'].dt.to_period('M')
bronx_evictions_monthly = bronx_evictions_copy.groupby('month').count()['primary_key']
bronx_monthly_df = bronx_evictions_monthly.reset_index()
bronx_monthly_df.columns = ['month', 'eviction_count']
bronx_monthly_df.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59
month,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
eviction_count,627,451,536,486,610,554,414,556,550,561,569,454,708,528,464,506,624,528,488,517,402,426,397,327,538,450,443,497,486,430,427,452,395,350,284,266,217,148,195,156,256,366,324,424,400,465,310,241,361,253,203,210,210,221,359,399,467,464,379,309


In [58]:
brooklyn_evictions_copy = brooklyn_evictions.copy()
brooklyn_evictions_copy['month'] = brooklyn_evictions_copy['executed_date'].dt.to_period('M')
brooklyn_evictions_monthly = brooklyn_evictions_copy.groupby('month').count()['primary_key']
brooklyn_monthly_df = brooklyn_evictions_monthly.reset_index()
brooklyn_monthly_df.columns = ['month', 'eviction_count']
brooklyn_monthly_df.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59
month,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
eviction_count,460,343,412,395,386,397,363,423,338,377,371,295,460,341,400,414,425,382,354,398,339,414,314,241,404,314,318,279,374,328,371,315,352,315,241,202,259,167,252,192,261,243,246,280,203,273,282,197,331,263,297,241,381,262,312,322,262,251,230,228


In [59]:
queens_evictions_copy = queens_evictions.copy()
queens_evictions_copy['month'] = queens_evictions_copy['executed_date'].dt.to_period('M')
queens_evictions_monthly = queens_evictions_copy.groupby('month').count()['primary_key']
queens_monthly_df = queens_evictions_monthly.reset_index()
queens_monthly_df.columns = ['month', 'eviction_count']
queens_monthly_df.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59
month,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
eviction_count,225,166,209,186,210,215,178,222,181,195,178,92,201,210,225,216,210,192,275,232,179,188,163,97,260,199,207,173,183,167,166,146,131,119,134,69,41,36,64,63,123,124,116,126,103,91,132,92,193,131,188,223,217,191,231,168,141,188,127,123


In [60]:
staten_evictions_copy = staten_evictions.copy()
staten_evictions_copy['month'] = staten_evictions_copy['executed_date'].dt.to_period('M')
staten_evictions_monthly = staten_evictions_copy.groupby('month').count()['primary_key']
staten_monthly_df = staten_evictions_monthly.reset_index()
staten_monthly_df.columns = ['month', 'eviction_count']
staten_monthly_df.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59
month,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
eviction_count,31,30,34,33,42,31,32,24,28,45,34,9,42,33,25,41,29,35,23,39,25,37,34,31,27,39,30,36,38,46,31,22,22,30,24,15,26,32,27,22,29,58,33,43,14,38,24,25,44,55,45,41,51,25,28,33,31,42,22,25


## **Step 2.2: Covid**

In [61]:
manhattan_evictions_copy_covid = manhattan_evictions_covid.copy()
manhattan_evictions_copy_covid['month'] = manhattan_evictions_copy_covid['executed_date'].dt.to_period('M')
manhattan_evictions_monthly_covid = manhattan_evictions_copy_covid.groupby('month').count()['primary_key']
man_monthly_df_covid = manhattan_evictions_monthly_covid.reset_index()
man_monthly_df_covid.columns = ['month', 'eviction_count']
man_monthly_df_covid.T
# 2020-04 to 2021-06 no evictions executed in Manhattan
# 16 months

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
month,2020-01,2020-02,2020-03,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12
eviction_count,166,113,67,7,5,1,3,1,1,9,13,18,18,34,40,38,57,57,82,90,67


In [92]:
brooklyn_evictions_copy_covid = brooklyn_evictions_covid.copy()
brooklyn_evictions_copy_covid['month'] = brooklyn_evictions_copy_covid['executed_date'].dt.to_period('M')
brooklyn_evictions_monthly_covid = brooklyn_evictions_copy_covid.groupby('month').count()['primary_key']
brooklyn_monthly_df_covid = brooklyn_evictions_monthly_covid.reset_index()
brooklyn_monthly_df_covid.columns = ['month', 'eviction_count']
brooklyn_monthly_df_covid.T
# 2020-04 to 2020-10, 2021-01 to 2021-03 no evictions executed in Brooklyn
# 10 months

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
month,2020-01,2020-02,2020-03,2020-11,2020-12,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12
eviction_count,293,232,125,1,2,2,3,2,10,3,4,2,9,9,18,39,70,84,93,92,88,125,127,147,192,102


In [94]:
bronx_evictions_copy_covid = bronx_evictions_covid.copy()
bronx_evictions_copy_covid['month'] = bronx_evictions_copy_covid['executed_date'].dt.to_period('M')
bronx_evictions_monthly_covid = bronx_evictions_copy_covid.groupby('month').count()['primary_key']
bronx_monthly_df_covid = bronx_evictions_monthly_covid.reset_index()
bronx_monthly_df_covid.columns = ['month', 'eviction_count']
bronx_monthly_df_covid.T
# 2020-04 to 2021-05, 2021-09, 2021-10 no evictons executed in Bronx
# 16 months no evictions in the Bronx

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
month,2020-01,2020-02,2020-03,2021-06,2021-07,2021-08,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12
eviction_count,393,297,127,1,2,6,2,2,27,33,29,28,59,55,56,59,93,167,161,88


In [65]:
queens_evictions_copy_covid = queens_evictions_covid.copy()
queens_evictions_copy_covid['month'] = queens_evictions_copy_covid['executed_date'].dt.to_period('M')
queens_evictions_monthly_covid = queens_evictions_copy_covid.groupby('month').count()['primary_key']
queens_monthly_df_covid = queens_evictions_monthly_covid.reset_index()
queens_monthly_df_covid.columns = ['month', 'eviction_count']
queens_monthly_df_covid.T
# 2020-04 to 2021-05, 2021-07, 2021-09 no evictions executed in Queens
# 16 months no evictions in queens

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
month,2020-01,2020-02,2020-03,2021-06,2021-08,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12
eviction_count,160,85,62,1,4,2,3,1,8,13,29,33,28,38,25,44,29,47,44,27


In [259]:
staten_evictions_copy_covid = staten_evictions_covid.copy()
staten_evictions_copy_covid['month'] = staten_evictions_copy_covid['executed_date'].dt.to_period('M')
staten_evictions_monthly_covid = staten_evictions_copy_covid.groupby('month').count()['primary_key']
staten_monthly_df_covid = staten_evictions_monthly_covid.reset_index()
staten_monthly_df_covid.columns = ['month', 'eviction_count']
staten_monthly_df_covid.T
# 2020-04 to 2020-11, 2021-01 to  2021-06, 2021-09, 2021-11 no evictions executed in si
# 16 months

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
month,2020-01,2020-02,2020-03,2020-12,2021-07,2021-08,2021-10,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12
eviction_count,28,16,12,3,3,5,1,3,11,9,13,11,12,9,11,13,12,15,30,20


In [70]:
man_monthly_df.month.unique()

<PeriodArray>
['2017-01', '2017-02', '2017-03', '2017-04', '2017-05', '2017-06', '2017-07',
 '2017-08', '2017-09', '2017-10', '2017-11', '2017-12', '2018-01', '2018-02',
 '2018-03', '2018-04', '2018-05', '2018-06', '2018-07', '2018-08', '2018-09',
 '2018-10', '2018-11', '2018-12', '2019-01', '2019-02', '2019-03', '2019-04',
 '2019-05', '2019-06', '2019-07', '2019-08', '2019-09', '2019-10', '2019-11',
 '2019-12', '2023-01', '2023-02', '2023-03', '2023-04', '2023-05', '2023-06',
 '2023-07', '2023-08', '2023-09', '2023-10', '2023-11', '2023-12', '2024-01',
 '2024-02', '2024-03', '2024-04', '2024-05', '2024-06', '2024-07', '2024-08',
 '2024-09', '2024-10', '2024-11', '2024-12']
Length: 60, dtype: period[M]

In [71]:
man_monthly_df_covid.month.unique()

<PeriodArray>
['2020-01', '2020-02', '2020-03', '2021-07', '2021-08', '2021-09', '2021-10',
 '2021-11', '2021-12', '2022-01', '2022-02', '2022-03', '2022-04', '2022-05',
 '2022-06', '2022-07', '2022-08', '2022-09', '2022-10', '2022-11', '2022-12']
Length: 21, dtype: period[M]

In [86]:
man_monthly_df.shape, man_monthly_df_covid.shape

((60, 2), (21, 2))

In [89]:
man_monthly_df_covid.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
month,2020-01,2020-02,2020-03,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12
eviction_count,166,113,67,7,5,1,3,1,1,9,13,18,18,34,40,38,57,57,82,90,67


In [91]:
combined_df_man = pd.concat([man_monthly_df, man_monthly_df_covid])
combined_df_man = combined_df_man.sort_values('month')
combined_df_man.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59
month,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
eviction_count,254,183,215,211,217,211,206,248,202,242,173,110,222,173,227,225,234,203,201,208,160,179,154,136,208,182,166,223,169,181,146,155,110,160,121,114,166,113,67,7,5,1,3,1,1,9,13,18,18,34,40,38,57,57,82,90,67,126,117,123,120,114,207,172,211,184,201,194,113,235,211,158,187,158,165,170,202,175,291,168,167


In [95]:
combined_df_brooklyn = pd.concat([brooklyn_monthly_df, brooklyn_monthly_df_covid])
combined_df_brooklyn = combined_df_brooklyn.sort_values('month')
combined_df_brooklyn.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59
month,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-11,2020-12,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
eviction_count,460,343,412,395,386,397,363,423,338,377,371,295,460,341,400,414,425,382,354,398,339,414,314,241,404,314,318,279,374,328,371,315,352,315,241,202,293,232,125,1,2,2,3,2,10,3,4,2,9,9,18,39,70,84,93,92,88,125,127,147,192,102,259,167,252,192,261,243,246,280,203,273,282,197,331,263,297,241,381,262,312,322,262,251,230,228


In [96]:
combined_df_bronx = pd.concat([bronx_monthly_df, bronx_monthly_df_covid])
combined_df_bronx = combined_df_bronx.sort_values('month')
combined_df_bronx.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59
month,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2021-06,2021-07,2021-08,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
eviction_count,627,451,536,486,610,554,414,556,550,561,569,454,708,528,464,506,624,528,488,517,402,426,397,327,538,450,443,497,486,430,427,452,395,350,284,266,393,297,127,1,2,6,2,2,27,33,29,28,59,55,56,59,93,167,161,88,217,148,195,156,256,366,324,424,400,465,310,241,361,253,203,210,210,221,359,399,467,464,379,309


In [97]:
combined_df_queens = pd.concat([queens_monthly_df, queens_monthly_df_covid])
combined_df_queens = combined_df_queens.sort_values('month')
combined_df_queens.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59
month,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2021-06,2021-08,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
eviction_count,225,166,209,186,210,215,178,222,181,195,178,92,201,210,225,216,210,192,275,232,179,188,163,97,260,199,207,173,183,167,166,146,131,119,134,69,160,85,62,1,4,2,3,1,8,13,29,33,28,38,25,44,29,47,44,27,41,36,64,63,123,124,116,126,103,91,132,92,193,131,188,223,217,191,231,168,141,188,127,123


In [98]:
combined_df_si = pd.concat([staten_monthly_df, staten_monthly_df_covid])
combined_df_si = combined_df_si.sort_values('month')
combined_df_si.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59
month,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-12,2021-07,2021-08,2021-10,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
eviction_count,31,30,34,33,42,31,32,24,28,45,34,9,42,33,25,41,29,35,23,39,25,37,34,31,27,39,30,36,38,46,31,22,22,30,24,15,28,16,12,3,3,5,1,3,11,9,13,11,12,9,11,13,12,15,30,20,26,32,27,22,29,58,33,43,14,38,24,25,44,55,45,41,51,25,28,33,31,42,22,25


In [ ]:
combined_df_all = pd.concat([])

In [102]:
combined_df_man.shape, combined_df_brooklyn.shape, combined_df_bronx.shape, combined_df_queens.shape, combined_df_si.shape

((81, 2), (86, 2), (80, 2), (80, 2), (80, 2))

# **Step 3: Now, add back the months with no evictions and fillna**

## **Step 3.1 Normal times**

### **Step 3.1.1Manhattan**

In [224]:
man_monthly_df['month'] = man_monthly_df['month'].dt.to_timestamp()
man_monthly_df= man_monthly_df.set_index('month')
man_monthly_df.head()

,eviction_count
month,
2017-01-01,254
2017-02-01,183
2017-03-01,215
2017-04-01,211
2017-05-01,217


In [225]:
all_months = pd.date_range(
    start=man_monthly_df.index.min(),
    end=man_monthly_df.index.max(),
    freq='MS'
)

In [226]:
man_df_complete = man_monthly_df.reindex(all_months)
man_df_complete['eviction_count'] = man_df_complete['eviction_count'].fillna(0)
man_df_complete['eviction_count'] = man_df_complete.eviction_count.astype(int)
man_df_complete.shape
# 3*12 = 36, correct

(96, 1)

In [227]:
man_df_complete.T

,2017-01-01,2017-02-01,2017-03-01,2017-04-01,2017-05-01,2017-06-01,2017-07-01,2017-08-01,2017-09-01,2017-10-01,2017-11-01,2017-12-01,2018-01-01,2018-02-01,2018-03-01,2018-04-01,2018-05-01,2018-06-01,2018-07-01,2018-08-01,2018-09-01,2018-10-01,2018-11-01,2018-12-01,2019-01-01,2019-02-01,2019-03-01,2019-04-01,2019-05-01,2019-06-01,2019-07-01,2019-08-01,2019-09-01,2019-10-01,2019-11-01,2019-12-01,2020-01-01,2020-02-01,2020-03-01,2020-04-01,2020-05-01,2020-06-01,2020-07-01,2020-08-01,2020-09-01,2020-10-01,2020-11-01,2020-12-01,2021-01-01,2021-02-01,2021-03-01,2021-04-01,2021-05-01,2021-06-01,2021-07-01,2021-08-01,2021-09-01,2021-10-01,2021-11-01,2021-12-01,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,2022-11-01,2022-12-01,2023-01-01,2023-02-01,2023-03-01,2023-04-01,2023-05-01,2023-06-01,2023-07-01,2023-08-01,2023-09-01,2023-10-01,2023-11-01,2023-12-01,2024-01-01,2024-02-01,2024-03-01,2024-04-01,2024-05-01,2024-06-01,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01
eviction_count,254,183,215,211,217,211,206,248,202,242,173,110,222,173,227,225,234,203,201,208,160,179,154,136,208,182,166,223,169,181,146,155,110,160,121,114,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,126,117,123,120,114,207,172,211,184,201,194,113,235,211,158,187,158,165,170,202,175,291,168,167


### **Step 3.1.2: Brooklyn**

In [128]:
combined_df_brooklyn['month'] = combined_df_brooklyn['month'].dt.to_timestamp()
combined_df_brooklyn = combined_df_brooklyn.set_index('month')
combined_df_brooklyn.head()

,eviction_count
month,
2017-01-01,460
2017-02-01,343
2017-03-01,412
2017-04-01,395
2017-05-01,386


In [129]:
all_months = pd.date_range(
    start=combined_df_brooklyn.index.min(),
    end=combined_df_brooklyn.index.max(),
    freq='MS'
)

In [149]:
bk_df_complete = combined_df_brooklyn.reindex(all_months)
bk_df_complete['eviction_count'] = bk_df_complete['eviction_count'].fillna(0)
bk_df_complete['eviction_count'] = bk_df_complete.eviction_count.astype(int)
bk_df_complete.T.shape

(1, 96)

In [150]:
bk_df_complete.T

,2017-01-01,2017-02-01,2017-03-01,2017-04-01,2017-05-01,2017-06-01,2017-07-01,2017-08-01,2017-09-01,2017-10-01,2017-11-01,2017-12-01,2018-01-01,2018-02-01,2018-03-01,2018-04-01,2018-05-01,2018-06-01,2018-07-01,2018-08-01,2018-09-01,2018-10-01,2018-11-01,2018-12-01,2019-01-01,2019-02-01,2019-03-01,2019-04-01,2019-05-01,2019-06-01,2019-07-01,2019-08-01,2019-09-01,2019-10-01,2019-11-01,2019-12-01,2020-01-01,2020-02-01,2020-03-01,2020-04-01,2020-05-01,2020-06-01,2020-07-01,2020-08-01,2020-09-01,2020-10-01,2020-11-01,2020-12-01,2021-01-01,2021-02-01,2021-03-01,2021-04-01,2021-05-01,2021-06-01,2021-07-01,2021-08-01,2021-09-01,2021-10-01,2021-11-01,2021-12-01,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,2022-11-01,2022-12-01,2023-01-01,2023-02-01,2023-03-01,2023-04-01,2023-05-01,2023-06-01,2023-07-01,2023-08-01,2023-09-01,2023-10-01,2023-11-01,2023-12-01,2024-01-01,2024-02-01,2024-03-01,2024-04-01,2024-05-01,2024-06-01,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01
eviction_count,460,343,412,395,386,397,363,423,338,377,371,295,460,341,400,414,425,382,354,398,339,414,314,241,404,314,318,279,374,328,371,315,352,315,241,202,293,232,125,0,0,0,0,0,0,0,1,2,0,0,0,2,3,2,10,3,4,2,9,9,18,39,70,84,93,92,88,125,127,147,192,102,259,167,252,192,261,243,246,280,203,273,282,197,331,263,297,241,381,262,312,322,262,251,230,228


### **Step 3.1.3 Bronx**

In [154]:
combined_df_bronx.T

month,2017-01-01,2017-02-01,2017-03-01,2017-04-01,2017-05-01,2017-06-01,2017-07-01,2017-08-01,2017-09-01,2017-10-01,2017-11-01,2017-12-01,2018-01-01,2018-02-01,2018-03-01,2018-04-01,2018-05-01,2018-06-01,2018-07-01,2018-08-01,2018-09-01,2018-10-01,2018-11-01,2018-12-01,2019-01-01,2019-02-01,2019-03-01,2019-04-01,2019-05-01,2019-06-01,2019-07-01,2019-08-01,2019-09-01,2019-10-01,2019-11-01,2019-12-01,2020-01-01,2020-02-01,2020-03-01,2021-06-01,2021-07-01,2021-08-01,2021-11-01,2021-12-01,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,2022-11-01,2022-12-01,2023-01-01,2023-02-01,2023-03-01,2023-04-01,2023-05-01,2023-06-01,2023-07-01,2023-08-01,2023-09-01,2023-10-01,2023-11-01,2023-12-01,2024-01-01,2024-02-01,2024-03-01,2024-04-01,2024-05-01,2024-06-01,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01
eviction_count,627,451,536,486,610,554,414,556,550,561,569,454,708,528,464,506,624,528,488,517,402,426,397,327,538,450,443,497,486,430,427,452,395,350,284,266,393,297,127,1,2,6,2,2,27,33,29,28,59,55,56,59,93,167,161,88,217,148,195,156,256,366,324,424,400,465,310,241,361,253,203,210,210,221,359,399,467,464,379,309


In [162]:
combined_df_bronx['month'] = combined_df_bronx['month'].dt.to_timestamp()
combined_df_bronx = combined_df_bronx.set_index('month')
combined_df_bronx.head()

In [156]:
all_months = pd.date_range(
    start=combined_df_bronx.index.min(),
    end=combined_df_bronx.index.max(),
    freq='MS'
)

In [157]:
br_df_complete = combined_df_bronx.reindex(all_months)
br_df_complete['eviction_count'] = br_df_complete['eviction_count'].fillna(0)
br_df_complete['eviction_count'] = br_df_complete.eviction_count.astype(int)
br_df_complete.T.shape

(1, 96)

In [158]:
br_df_complete.T

,2017-01-01,2017-02-01,2017-03-01,2017-04-01,2017-05-01,2017-06-01,2017-07-01,2017-08-01,2017-09-01,2017-10-01,2017-11-01,2017-12-01,2018-01-01,2018-02-01,2018-03-01,2018-04-01,2018-05-01,2018-06-01,2018-07-01,2018-08-01,2018-09-01,2018-10-01,2018-11-01,2018-12-01,2019-01-01,2019-02-01,2019-03-01,2019-04-01,2019-05-01,2019-06-01,2019-07-01,2019-08-01,2019-09-01,2019-10-01,2019-11-01,2019-12-01,2020-01-01,2020-02-01,2020-03-01,2020-04-01,2020-05-01,2020-06-01,2020-07-01,2020-08-01,2020-09-01,2020-10-01,2020-11-01,2020-12-01,2021-01-01,2021-02-01,2021-03-01,2021-04-01,2021-05-01,2021-06-01,2021-07-01,2021-08-01,2021-09-01,2021-10-01,2021-11-01,2021-12-01,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,2022-11-01,2022-12-01,2023-01-01,2023-02-01,2023-03-01,2023-04-01,2023-05-01,2023-06-01,2023-07-01,2023-08-01,2023-09-01,2023-10-01,2023-11-01,2023-12-01,2024-01-01,2024-02-01,2024-03-01,2024-04-01,2024-05-01,2024-06-01,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01
eviction_count,627,451,536,486,610,554,414,556,550,561,569,454,708,528,464,506,624,528,488,517,402,426,397,327,538,450,443,497,486,430,427,452,395,350,284,266,393,297,127,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,6,0,0,2,2,27,33,29,28,59,55,56,59,93,167,161,88,217,148,195,156,256,366,324,424,400,465,310,241,361,253,203,210,210,221,359,399,467,464,379,309


### **Step 3.1.4 Queens**

In [163]:
combined_df_queens['month'] = combined_df_queens['month'].dt.to_timestamp()
combined_df_queens = combined_df_queens.set_index('month')
combined_df_queens.head()

In [164]:
all_months = pd.date_range(
    start=combined_df_queens.index.min(),
    end=combined_df_queens.index.max(),
    freq='MS'
)

In [165]:
queens_df_complete = combined_df_queens.reindex(all_months)
queens_df_complete['eviction_count'] = queens_df_complete['eviction_count'].fillna(0)
queens_df_complete['eviction_count'] = queens_df_complete.eviction_count.astype(int)
queens_df_complete.T.shape

(1, 96)

In [166]:
queens_df_complete.T

,2017-01-01,2017-02-01,2017-03-01,2017-04-01,2017-05-01,2017-06-01,2017-07-01,2017-08-01,2017-09-01,2017-10-01,2017-11-01,2017-12-01,2018-01-01,2018-02-01,2018-03-01,2018-04-01,2018-05-01,2018-06-01,2018-07-01,2018-08-01,2018-09-01,2018-10-01,2018-11-01,2018-12-01,2019-01-01,2019-02-01,2019-03-01,2019-04-01,2019-05-01,2019-06-01,2019-07-01,2019-08-01,2019-09-01,2019-10-01,2019-11-01,2019-12-01,2020-01-01,2020-02-01,2020-03-01,2020-04-01,2020-05-01,2020-06-01,2020-07-01,2020-08-01,2020-09-01,2020-10-01,2020-11-01,2020-12-01,2021-01-01,2021-02-01,2021-03-01,2021-04-01,2021-05-01,2021-06-01,2021-07-01,2021-08-01,2021-09-01,2021-10-01,2021-11-01,2021-12-01,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,2022-11-01,2022-12-01,2023-01-01,2023-02-01,2023-03-01,2023-04-01,2023-05-01,2023-06-01,2023-07-01,2023-08-01,2023-09-01,2023-10-01,2023-11-01,2023-12-01,2024-01-01,2024-02-01,2024-03-01,2024-04-01,2024-05-01,2024-06-01,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01
eviction_count,225,166,209,186,210,215,178,222,181,195,178,92,201,210,225,216,210,192,275,232,179,188,163,97,260,199,207,173,183,167,166,146,131,119,134,69,160,85,62,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,4,0,2,3,1,8,13,29,33,28,38,25,44,29,47,44,27,41,36,64,63,123,124,116,126,103,91,132,92,193,131,188,223,217,191,231,168,141,188,127,123


### **Step 3.1.5 Staten Island**

In [ ]:
combined_df_si['month'] = combined_df_si['month'].dt.to_timestamp()
combined_df_si = combined_df_si.set_index('month')
combined_df_si.head()

In [167]:
all_months = pd.date_range(
    start=combined_df_si.index.min(),
    end=combined_df_si.index.max(),
    freq='MS'
)

# staten_monthly_df_covid

In [168]:
si_df_complete = combined_df_si.reindex(all_months)
si_df_complete['eviction_count'] = si_df_complete['eviction_count'].fillna(0)
si_df_complete['eviction_count'] = si_df_complete.eviction_count.astype(int)
si_df_complete.T.shape

(1, 96)

In [190]:
si_df_complete.T

,2017-01-01,2017-02-01,2017-03-01,2017-04-01,2017-05-01,2017-06-01,2017-07-01,2017-08-01,2017-09-01,2017-10-01,2017-11-01,2017-12-01,2018-01-01,2018-02-01,2018-03-01,2018-04-01,2018-05-01,2018-06-01,2018-07-01,2018-08-01,2018-09-01,2018-10-01,2018-11-01,2018-12-01,2019-01-01,2019-02-01,2019-03-01,2019-04-01,2019-05-01,2019-06-01,2019-07-01,2019-08-01,2019-09-01,2019-10-01,2019-11-01,2019-12-01,2020-01-01,2020-02-01,2020-03-01,2020-04-01,2020-05-01,2020-06-01,2020-07-01,2020-08-01,2020-09-01,2020-10-01,2020-11-01,2020-12-01,2021-01-01,2021-02-01,2021-03-01,2021-04-01,2021-05-01,2021-06-01,2021-07-01,2021-08-01,2021-09-01,2021-10-01,2021-11-01,2021-12-01,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,2022-11-01,2022-12-01,2023-01-01,2023-02-01,2023-03-01,2023-04-01,2023-05-01,2023-06-01,2023-07-01,2023-08-01,2023-09-01,2023-10-01,2023-11-01,2023-12-01,2024-01-01,2024-02-01,2024-03-01,2024-04-01,2024-05-01,2024-06-01,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01
eviction_count,31,30,34,33,42,31,32,24,28,45,34,9,42,33,25,41,29,35,23,39,25,37,34,31,27,39,30,36,38,46,31,22,22,30,24,15,28,16,12,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,3,5,0,1,0,3,11,9,13,11,12,9,11,13,12,15,30,20,26,32,27,22,29,58,33,43,14,38,24,25,44,55,45,41,51,25,28,33,31,42,22,25


## **Step 3.2: covid times**

### **Step 3.2.1 Manhattan**

In [230]:
man_monthly_df_covid['month'] = man_monthly_df_covid['month'].dt.to_timestamp()
man_monthly_df_covid = man_monthly_df_covid.set_index('month')
man_monthly_df_covid.head()

,eviction_count
month,
2020-01-01,166
2020-02-01,113
2020-03-01,67
2021-07-01,7
2021-08-01,5


In [231]:
all_months = pd.date_range(
    start=man_monthly_df_covid.index.min(),
    end=man_monthly_df_covid.index.max(),
    freq='MS'
)

In [232]:
man_df_complete_covid = man_monthly_df_covid.reindex(all_months)
man_df_complete_covid['eviction_count'] = man_df_complete_covid['eviction_count'].fillna(0)
man_df_complete_covid['eviction_count'] = man_df_complete_covid.eviction_count.astype(int)
man_df_complete_covid.shape
# 3*12 = 36, correct

(36, 1)

In [233]:
man_df_complete_covid.T

,2020-01-01,2020-02-01,2020-03-01,2020-04-01,2020-05-01,2020-06-01,2020-07-01,2020-08-01,2020-09-01,2020-10-01,2020-11-01,2020-12-01,2021-01-01,2021-02-01,2021-03-01,2021-04-01,2021-05-01,2021-06-01,2021-07-01,2021-08-01,2021-09-01,2021-10-01,2021-11-01,2021-12-01,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,2022-11-01,2022-12-01
eviction_count,166,113,67,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,5,1,3,1,1,9,13,18,18,34,40,38,57,57,82,90,67


### **Step 3.2.2. Brooklyn**

In [234]:
brooklyn_monthly_df_covid['month'] = brooklyn_monthly_df_covid['month'].dt.to_timestamp()
brooklyn_monthly_df_covid = brooklyn_monthly_df_covid.set_index('month')
brooklyn_monthly_df_covid.head()

,eviction_count
month,
2020-01-01,293
2020-02-01,232
2020-03-01,125
2020-11-01,1
2020-12-01,2


In [235]:
all_months = pd.date_range(
    start=brooklyn_monthly_df_covid.index.min(),
    end=brooklyn_monthly_df_covid.index.max(),
    freq='MS'
)

In [236]:
bk_df_complete_covid = brooklyn_monthly_df_covid.reindex(all_months)
bk_df_complete_covid['eviction_count'] = bk_df_complete_covid['eviction_count'].fillna(0)
bk_df_complete_covid['eviction_count'] = bk_df_complete_covid.eviction_count.astype(int)
bk_df_complete_covid.T.shape

(1, 36)

In [237]:
bk_df_complete_covid.T

,2020-01-01,2020-02-01,2020-03-01,2020-04-01,2020-05-01,2020-06-01,2020-07-01,2020-08-01,2020-09-01,2020-10-01,2020-11-01,2020-12-01,2021-01-01,2021-02-01,2021-03-01,2021-04-01,2021-05-01,2021-06-01,2021-07-01,2021-08-01,2021-09-01,2021-10-01,2021-11-01,2021-12-01,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,2022-11-01,2022-12-01
eviction_count,293,232,125,0,0,0,0,0,0,0,1,2,0,0,0,2,3,2,10,3,4,2,9,9,18,39,70,84,93,92,88,125,127,147,192,102


### **3.2.3 Bronx**

In [238]:
bronx_monthly_df_covid['month'] = bronx_monthly_df_covid['month'].dt.to_timestamp()
bronx_monthly_df_covid = bronx_monthly_df_covid.set_index('month')
bronx_monthly_df_covid.head()

,eviction_count
month,
2020-01-01,393
2020-02-01,297
2020-03-01,127
2021-06-01,1
2021-07-01,2


In [239]:
all_months = pd.date_range(
    start=bronx_monthly_df_covid.index.min(),
    end=bronx_monthly_df_covid.index.max(),
    freq='MS'
)

In [240]:
br_df_complete_covid = bronx_monthly_df_covid.reindex(all_months)
br_df_complete_covid['eviction_count'] = br_df_complete_covid['eviction_count'].fillna(0)
br_df_complete_covid['eviction_count'] = br_df_complete_covid.eviction_count.astype(int)
br_df_complete_covid.T.shape

(1, 36)

In [241]:
br_df_complete_covid.T

,2020-01-01,2020-02-01,2020-03-01,2020-04-01,2020-05-01,2020-06-01,2020-07-01,2020-08-01,2020-09-01,2020-10-01,2020-11-01,2020-12-01,2021-01-01,2021-02-01,2021-03-01,2021-04-01,2021-05-01,2021-06-01,2021-07-01,2021-08-01,2021-09-01,2021-10-01,2021-11-01,2021-12-01,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,2022-11-01,2022-12-01
eviction_count,393,297,127,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,6,0,0,2,2,27,33,29,28,59,55,56,59,93,167,161,88


### **3.2.4 Queens**

In [242]:
queens_monthly_df_covid['month'] = queens_monthly_df_covid['month'].dt.to_timestamp()
queens_monthly_df_covid = queens_monthly_df_covid.set_index('month')
queens_monthly_df_covid.head()

,eviction_count
month,
2020-01-01,160
2020-02-01,85
2020-03-01,62
2021-06-01,1
2021-08-01,4


In [243]:
all_months = pd.date_range(
    start=queens_monthly_df_covid.index.min(),
    end=queens_monthly_df_covid.index.max(),
    freq='MS'
)

In [244]:
queens_df_complete_covid = queens_monthly_df_covid.reindex(all_months)
queens_df_complete_covid['eviction_count'] = queens_df_complete_covid['eviction_count'].fillna(0)
queens_df_complete_covid['eviction_count'] = queens_df_complete_covid.eviction_count.astype(int)
queens_df_complete_covid.T.shape

(1, 36)

In [245]:
queens_df_complete_covid.T

,2020-01-01,2020-02-01,2020-03-01,2020-04-01,2020-05-01,2020-06-01,2020-07-01,2020-08-01,2020-09-01,2020-10-01,2020-11-01,2020-12-01,2021-01-01,2021-02-01,2021-03-01,2021-04-01,2021-05-01,2021-06-01,2021-07-01,2021-08-01,2021-09-01,2021-10-01,2021-11-01,2021-12-01,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,2022-11-01,2022-12-01
eviction_count,160,85,62,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,4,0,2,3,1,8,13,29,33,28,38,25,44,29,47,44,27


### **3.2.5 Staten Island**

In [260]:
staten_monthly_df_covid['month'] = staten_monthly_df_covid['month'].dt.to_timestamp()
staten_monthly_df_covid = staten_monthly_df_covid.set_index('month')
staten_monthly_df_covid.head()

,eviction_count
month,
2020-01-01,28
2020-02-01,16
2020-03-01,12
2020-12-01,3
2021-07-01,3


In [262]:
all_months = pd.date_range(
    start=staten_monthly_df_covid.index.min(),
    end=staten_monthly_df_covid.index.max(),
    freq='MS'
)

In [263]:
si_df_complete_covid = combined_df_si.reindex(all_months)
si_df_complete_covid['eviction_count'] = si_df_complete_covid['eviction_count'].fillna(0)
si_df_complete_covid['eviction_count'] = si_df_complete_covid.eviction_count.astype(int)
si_df_complete_covid.T.shape


(1, 36)

In [264]:
si_df_complete_covid.T

,2020-01-01,2020-02-01,2020-03-01,2020-04-01,2020-05-01,2020-06-01,2020-07-01,2020-08-01,2020-09-01,2020-10-01,2020-11-01,2020-12-01,2021-01-01,2021-02-01,2021-03-01,2021-04-01,2021-05-01,2021-06-01,2021-07-01,2021-08-01,2021-09-01,2021-10-01,2021-11-01,2021-12-01,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,2022-11-01,2022-12-01
eviction_count,28,16,12,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,3,5,0,1,0,3,11,9,13,11,12,9,11,13,12,15,30,20


# **Step 4: Rent Eviction time series**

## **Step 4.1 Normal Times**

In [188]:
si_df_complete.shape, queens_df_complete.shape, br_df_complete.shape, bk_df_complete.shape, man_df_complete.shape

((96, 1), (96, 1), (96, 1), (96, 1), (96, 1))

In [186]:
br_rent.rename(columns={'date': 'month'}, inplace=True)
bk_rent.rename(columns={'date': 'month'}, inplace=True)
man_rent.rename(columns={'date': 'month'}, inplace=True)
queens_rent.rename(columns={'date': 'month'}, inplace=True)
si_rent.rename(columns={'date': 'month'}, inplace=True)

In [191]:
si_df_complete.reset_index(inplace=True)
queens_df_complete.reset_index(inplace=True)
br_df_complete.reset_index(inplace=True)
bk_df_complete.reset_index(inplace=True)
man_df_complete.reset_index(inplace=True)

In [192]:
si_df_complete.shape, queens_df_complete.shape, br_df_complete.shape, bk_df_complete.shape, man_df_complete.shape

((96, 2), (96, 2), (96, 2), (96, 2), (96, 2))

In [196]:
si_df_complete.rename(columns={'index': 'month'}, inplace=True)
queens_df_complete.rename(columns={'index': 'month'}, inplace=True)
br_df_complete.rename(columns={'index': 'month'}, inplace=True)
bk_df_complete.rename(columns={'index': 'month'}, inplace=True)
man_df_complete.rename(columns={'index': 'month'}, inplace=True)

In [199]:
br_rent.shape, bk_rent.shape, man_rent.shape, queens_rent.shape, si_rent.shape, \
br_rent.columns, bk_rent.columns, man_rent.columns, queens_rent.columns, si_rent.columns, \
si_df_complete.shape, queens_df_complete.shape, br_df_complete.shape, bk_df_complete.shape, man_df_complete.shape, \
si_df_complete.columns, queens_df_complete.columns, br_df_complete.columns, bk_df_complete.columns, man_df_complete.columns

((96, 2),
 (96, 2),
 (96, 2),
 (96, 2),
 (96, 2),
 Index(['month', 'median_price'], dtype='object'),
 Index(['month', 'median_price'], dtype='object'),
 Index(['month', 'median_price'], dtype='object'),
 Index(['month', 'median_price'], dtype='object'),
 Index(['month', 'median_price'], dtype='object'),
 (96, 2),
 (96, 2),
 (96, 2),
 (96, 2),
 (96, 2),
 Index(['month', 'eviction_count'], dtype='object'),
 Index(['month', 'eviction_count'], dtype='object'),
 Index(['month', 'eviction_count'], dtype='object'),
 Index(['month', 'eviction_count'], dtype='object'),
 Index(['month', 'eviction_count'], dtype='object'))

In [205]:
si_rent.month = si_rent.month.astype(str)
queens_rent.month = queens_rent.month.astype(str)
br_rent.month = br_rent.month.astype(str)
bk_rent.month = bk_rent.month.astype(str)
man_rent.month = man_rent.month.astype(str)
si_df_complete.month = si_df_complete.month.astype(str)
queens_df_complete.month = queens_df_complete.month.astype(str)
br_df_complete.month = br_df_complete.month.astype(str)
bk_df_complete.month = bk_df_complete.month.astype(str)
man_df_complete.month = man_df_complete.month.astype(str)

In [206]:
man_rent_evi = man_rent.merge(man_df_complete, on='month', how='left')
man_rent_evi.head()

,month,median_price,eviction_count
0,2017-01-01,3299.0000,254
1,2017-02-01,3310.5000,183
2,2017-03-01,3265.0000,215
3,2017-04-01,3349.0000,211
4,2017-05-01,3371.5000,217


In [207]:
man_rent_evi.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95
month,2017-01-01,2017-02-01,2017-03-01,2017-04-01,2017-05-01,2017-06-01,2017-07-01,2017-08-01,2017-09-01,2017-10-01,2017-11-01,2017-12-01,2018-01-01,2018-02-01,2018-03-01,2018-04-01,2018-05-01,2018-06-01,2018-07-01,2018-08-01,2018-09-01,2018-10-01,2018-11-01,2018-12-01,2019-01-01,2019-02-01,2019-03-01,2019-04-01,2019-05-01,2019-06-01,2019-07-01,2019-08-01,2019-09-01,2019-10-01,2019-11-01,2019-12-01,2020-01-01,2020-02-01,2020-03-01,2020-04-01,2020-05-01,2020-06-01,2020-07-01,2020-08-01,2020-09-01,2020-10-01,2020-11-01,2020-12-01,2021-01-01,2021-02-01,2021-03-01,2021-04-01,2021-05-01,2021-06-01,2021-07-01,2021-08-01,2021-09-01,2021-10-01,2021-11-01,2021-12-01,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,2022-11-01,2022-12-01,2023-01-01,2023-02-01,2023-03-01,2023-04-01,2023-05-01,2023-06-01,2023-07-01,2023-08-01,2023-09-01,2023-10-01,2023-11-01,2023-12-01,2024-01-01,2024-02-01,2024-03-01,2024-04-01,2024-05-01,2024-06-01,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01
median_price,3299.0000,3310.5000,3265.0000,3349.0000,3371.5000,3375.0000,3350.0000,3375.0000,3354.0000,3344.5000,3250.0000,3300.0000,3323.5000,3290.0000,3250.0000,3290.0000,3422.5000,3428.0000,3425.0000,3400.0000,3395.0000,3395.0000,3350.0000,3409.0000,3395.0000,3350.0000,3395.0000,3453.0000,3537.5000,3597.5000,3595.0000,3563.0000,3602.0000,3600.0000,3650.0000,3608.5000,3650.0000,3604.5000,3543.0000,3475.0000,3333.5000,3201.5000,3150.0000,3000.0000,2928.0000,2897.5000,2850.0000,2822.0000,2849.5000,2850.0000,2850.0000,2945.0000,3000.0000,3129.0000,3387.5000,3495.0000,3560.0000,3675.0000,3800.0000,3950.0000,3995.0000,4000.0000,4125.0000,4247.0000,4272.5000,4227.5000,4350.0000,4273.0000,4350.0000,4264.5000,4235.0000,4292.5000,4300.0000,4313.0000,4398.0000,4456.5000,4427.5000,4400.0000,4447.5000,4426.5000,4425.0000,4375.0000,4336.0000,4250.0000,4277.5000,4297.5000,4399.0000,4495.0000,4500.0000,4497.5000,4487.5000,4450.0000,4394.5000,4375.0000,4295.0000,4407.5000
eviction_count,254,183,215,211,217,211,206,248,202,242,173,110,222,173,227,225,234,203,201,208,160,179,154,136,208,182,166,223,169,181,146,155,110,160,121,114,166,113,67,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,5,1,3,1,1,9,13,18,18,34,40,38,57,57,82,90,67,126,117,123,120,114,207,172,211,184,201,194,113,235,211,158,187,158,165,170,202,175,291,168,167


In [208]:
bk_rent_evi = bk_rent.merge(bk_df_complete, on='month', how='left')
bk_rent_evi.head()

,month,median_price,eviction_count
0,2017-01-01,2295.0000,460
1,2017-02-01,2300.0000,343
2,2017-03-01,2300.0000,412
3,2017-04-01,2300.0000,395
4,2017-05-01,2300.0000,386


In [209]:
bk_rent_evi.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95
month,2017-01-01,2017-02-01,2017-03-01,2017-04-01,2017-05-01,2017-06-01,2017-07-01,2017-08-01,2017-09-01,2017-10-01,2017-11-01,2017-12-01,2018-01-01,2018-02-01,2018-03-01,2018-04-01,2018-05-01,2018-06-01,2018-07-01,2018-08-01,2018-09-01,2018-10-01,2018-11-01,2018-12-01,2019-01-01,2019-02-01,2019-03-01,2019-04-01,2019-05-01,2019-06-01,2019-07-01,2019-08-01,2019-09-01,2019-10-01,2019-11-01,2019-12-01,2020-01-01,2020-02-01,2020-03-01,2020-04-01,2020-05-01,2020-06-01,2020-07-01,2020-08-01,2020-09-01,2020-10-01,2020-11-01,2020-12-01,2021-01-01,2021-02-01,2021-03-01,2021-04-01,2021-05-01,2021-06-01,2021-07-01,2021-08-01,2021-09-01,2021-10-01,2021-11-01,2021-12-01,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,2022-11-01,2022-12-01,2023-01-01,2023-02-01,2023-03-01,2023-04-01,2023-05-01,2023-06-01,2023-07-01,2023-08-01,2023-09-01,2023-10-01,2023-11-01,2023-12-01,2024-01-01,2024-02-01,2024-03-01,2024-04-01,2024-05-01,2024-06-01,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01
median_price,2295.0000,2300.0000,2300.0000,2300.0000,2300.0000,2374.5000,2450.0000,2412.5000,2450.5000,2400.0000,2390.0000,2350.0000,2300.0000,2300.0000,2399.5000,2437.5000,2500.0000,2500.0000,2567.0000,2492.5000,2550.0000,2400.0000,2400.0000,2400.0000,2250.0000,2399.0000,2400.0000,2422.5000,2525.0000,2500.0000,2500.0000,2600.0000,2398.0000,2544.5000,2500.0000,2475.0000,2525.0000,2525.0000,2550.0000,2539.0000,2525.0000,2499.5000,2499.0000,2475.0000,2400.0000,2300.0000,2300.0000,2251.0000,2250.0000,2280.0000,2269.0000,2300.0000,2213.0000,2400.0000,2425.0000,2415.0000,2424.5000,2450.0000,2534.0000,2619.5000,2533.0000,2600.0000,2651.0000,2825.0000,2997.5000,2956.5000,3200.0000,3200.0000,3000.0000,3000.0000,2975.0000,2975.5000,3000.0000,3000.0000,3000.0000,3072.5000,3075.0000,3175.0000,3100.0000,3200.0000,3100.0000,3150.0000,3000.0000,3000.0000,3000.0000,3080.0000,3052.5000,3200.0000,3299.0000,3300.0000,3315.5000,3300.0000,3300.0000,3250.0000,3222.5000,3200.0000
eviction_count,460,343,412,395,386,397,363,423,338,377,371,295,460,341,400,414,425,382,354,398,339,414,314,241,404,314,318,279,374,328,371,315,352,315,241,202,293,232,125,0,0,0,0,0,0,0,1,2,0,0,0,2,3,2,10,3,4,2,9,9,18,39,70,84,93,92,88,125,127,147,192,102,259,167,252,192,261,243,246,280,203,273,282,197,331,263,297,241,381,262,312,322,262,251,230,228


In [210]:
br_rent_evi = br_rent.merge(br_df_complete, on='month', how='left')
br_rent_evi.head()

,month,median_price,eviction_count
0,2017-01-01,1510.0000,627
1,2017-02-01,1612.5000,451
2,2017-03-01,1700.0000,536
3,2017-04-01,1675.0000,486
4,2017-05-01,1680.0000,610


In [211]:
br_rent_evi.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95
month,2017-01-01,2017-02-01,2017-03-01,2017-04-01,2017-05-01,2017-06-01,2017-07-01,2017-08-01,2017-09-01,2017-10-01,2017-11-01,2017-12-01,2018-01-01,2018-02-01,2018-03-01,2018-04-01,2018-05-01,2018-06-01,2018-07-01,2018-08-01,2018-09-01,2018-10-01,2018-11-01,2018-12-01,2019-01-01,2019-02-01,2019-03-01,2019-04-01,2019-05-01,2019-06-01,2019-07-01,2019-08-01,2019-09-01,2019-10-01,2019-11-01,2019-12-01,2020-01-01,2020-02-01,2020-03-01,2020-04-01,2020-05-01,2020-06-01,2020-07-01,2020-08-01,2020-09-01,2020-10-01,2020-11-01,2020-12-01,2021-01-01,2021-02-01,2021-03-01,2021-04-01,2021-05-01,2021-06-01,2021-07-01,2021-08-01,2021-09-01,2021-10-01,2021-11-01,2021-12-01,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,2022-11-01,2022-12-01,2023-01-01,2023-02-01,2023-03-01,2023-04-01,2023-05-01,2023-06-01,2023-07-01,2023-08-01,2023-09-01,2023-10-01,2023-11-01,2023-12-01,2024-01-01,2024-02-01,2024-03-01,2024-04-01,2024-05-01,2024-06-01,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01
median_price,1510.0000,1612.5000,1700.0000,1675.0000,1680.0000,1650.0000,1650.0000,1700.0000,1700.0000,1700.0000,1716.5000,1613.0000,1691.5000,1718.0000,1737.5000,1800.0000,1800.0000,1819.0000,1785.0000,1800.0000,1800.0000,1898.0000,1750.0000,1790.0000,1806.0000,1800.0000,1800.0000,1775.0000,1800.0000,1850.0000,1874.0000,1862.5000,1899.0000,1862.0000,1884.0000,1872.5000,1838.0000,1874.5000,1900.0000,1952.0000,1860.0000,1875.0000,1900.0000,1794.0000,1777.0000,1835.0000,1849.0000,1725.0000,1825.0000,1770.5000,1847.5000,1825.0000,1899.0000,1850.0000,1913.5000,1900.0000,1888.0000,1937.5000,1995.0000,1896.0000,1800.0000,1825.0000,1895.0000,1887.5000,1950.0000,2100.0000,1995.0000,2200.0000,1991.0000,1950.0000,1950.0000,2051.5000,2100.0000,2174.0000,2175.0000,2187.5000,2150.0000,2349.0000,2242.5000,2237.5000,2311.5000,2300.0000,2299.0000,2150.0000,2250.0000,2386.5000,2162.0000,2374.0000,2383.0000,2150.0000,2599.0000,2500.0000,2412.5000,2395.0000,2495.0000,2362.5000
eviction_count,627,451,536,486,610,554,414,556,550,561,569,454,708,528,464,506,624,528,488,517,402,426,397,327,538,450,443,497,486,430,427,452,395,350,284,266,393,297,127,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,6,0,0,2,2,27,33,29,28,59,55,56,59,93,167,161,88,217,148,195,156,256,366,324,424,400,465,310,241,361,253,203,210,210,221,359,399,467,464,379,309


In [212]:
queens_rent_evi = queens_rent.merge(queens_df_complete, on='month', how='left')
queens_rent_evi.head()

,month,median_price,eviction_count
0,2017-01-01,1995.0000,225
1,2017-02-01,2037.0000,166
2,2017-03-01,1987.5000,209
3,2017-04-01,2000.0000,186
4,2017-05-01,2000.0000,210


In [213]:
queens_rent_evi.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95
month,2017-01-01,2017-02-01,2017-03-01,2017-04-01,2017-05-01,2017-06-01,2017-07-01,2017-08-01,2017-09-01,2017-10-01,2017-11-01,2017-12-01,2018-01-01,2018-02-01,2018-03-01,2018-04-01,2018-05-01,2018-06-01,2018-07-01,2018-08-01,2018-09-01,2018-10-01,2018-11-01,2018-12-01,2019-01-01,2019-02-01,2019-03-01,2019-04-01,2019-05-01,2019-06-01,2019-07-01,2019-08-01,2019-09-01,2019-10-01,2019-11-01,2019-12-01,2020-01-01,2020-02-01,2020-03-01,2020-04-01,2020-05-01,2020-06-01,2020-07-01,2020-08-01,2020-09-01,2020-10-01,2020-11-01,2020-12-01,2021-01-01,2021-02-01,2021-03-01,2021-04-01,2021-05-01,2021-06-01,2021-07-01,2021-08-01,2021-09-01,2021-10-01,2021-11-01,2021-12-01,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,2022-11-01,2022-12-01,2023-01-01,2023-02-01,2023-03-01,2023-04-01,2023-05-01,2023-06-01,2023-07-01,2023-08-01,2023-09-01,2023-10-01,2023-11-01,2023-12-01,2024-01-01,2024-02-01,2024-03-01,2024-04-01,2024-05-01,2024-06-01,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01
median_price,1995.0000,2037.0000,1987.5000,2000.0000,2000.0000,2150.0000,2100.0000,2000.0000,1995.0000,1997.0000,1981.5000,1992.0000,2000.0000,1950.0000,2000.0000,2036.5000,1985.0000,2050.0000,2081.5000,2072.5000,2000.0000,2000.0000,2000.0000,2000.0000,2000.0000,2000.0000,2012.0000,2000.0000,1999.0000,2050.0000,2100.0000,2050.0000,2100.0000,2075.0000,2050.0000,2000.0000,2000.0000,2100.0000,2162.5000,2100.0000,2036.5000,2000.0000,2051.5000,2000.0000,2000.0000,1978.0000,2000.0000,1995.0000,1925.0000,1950.0000,1937.5000,1950.0000,2000.0000,2025.0000,2000.0000,2050.0000,2100.0000,2100.0000,2100.0000,2064.0000,2150.0000,2174.5000,2200.0000,2295.0000,2300.0000,2400.0000,2350.0000,2405.5000,2400.0000,2400.0000,2399.5000,2387.0000,2362.5000,2499.0000,2495.0000,2450.0000,2475.0000,2496.5000,2499.5000,2600.0000,2560.0000,2599.0000,2574.5000,2500.0000,2592.0000,2511.5000,2588.0000,2657.0000,2650.0000,2687.0000,2624.0000,2650.0000,2598.0000,2675.0000,2640.0000,2650.0000
eviction_count,225,166,209,186,210,215,178,222,181,195,178,92,201,210,225,216,210,192,275,232,179,188,163,97,260,199,207,173,183,167,166,146,131,119,134,69,160,85,62,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,4,0,2,3,1,8,13,29,33,28,38,25,44,29,47,44,27,41,36,64,63,123,124,116,126,103,91,132,92,193,131,188,223,217,191,231,168,141,188,127,123


In [214]:
si_rent_evi = si_rent.merge(si_df_complete, on='month', how='left')
si_rent_evi.head()

,month,median_price,eviction_count
0,2017-01-01,1950.0000,31
1,2017-02-01,1900.0000,30
2,2017-03-01,1900.0000,34
3,2017-04-01,1918.0000,33
4,2017-05-01,2000.0000,42


In [215]:
si_rent_evi.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95
month,2017-01-01,2017-02-01,2017-03-01,2017-04-01,2017-05-01,2017-06-01,2017-07-01,2017-08-01,2017-09-01,2017-10-01,2017-11-01,2017-12-01,2018-01-01,2018-02-01,2018-03-01,2018-04-01,2018-05-01,2018-06-01,2018-07-01,2018-08-01,2018-09-01,2018-10-01,2018-11-01,2018-12-01,2019-01-01,2019-02-01,2019-03-01,2019-04-01,2019-05-01,2019-06-01,2019-07-01,2019-08-01,2019-09-01,2019-10-01,2019-11-01,2019-12-01,2020-01-01,2020-02-01,2020-03-01,2020-04-01,2020-05-01,2020-06-01,2020-07-01,2020-08-01,2020-09-01,2020-10-01,2020-11-01,2020-12-01,2021-01-01,2021-02-01,2021-03-01,2021-04-01,2021-05-01,2021-06-01,2021-07-01,2021-08-01,2021-09-01,2021-10-01,2021-11-01,2021-12-01,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,2022-11-01,2022-12-01,2023-01-01,2023-02-01,2023-03-01,2023-04-01,2023-05-01,2023-06-01,2023-07-01,2023-08-01,2023-09-01,2023-10-01,2023-11-01,2023-12-01,2024-01-01,2024-02-01,2024-03-01,2024-04-01,2024-05-01,2024-06-01,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01
median_price,1950.0000,1900.0000,1900.0000,1918.0000,2000.0000,2033.0000,1950.0000,1950.0000,1790.0000,1875.0000,1900.0000,1925.0000,1800.0000,1842.0000,1800.0000,1825.0000,1829.0000,1894.0000,1950.0000,2000.0000,2000.0000,1865.0000,1959.0000,1865.0000,1853.0000,1850.0000,1890.0000,1875.0000,2050.0000,2050.0000,2000.0000,2045.0000,2200.0000,2200.0000,2200.0000,2000.0000,1952.0000,1998.0000,2123.0000,1831.0000,1833.0000,1900.0000,1900.0000,1850.0000,1863.0000,1950.0000,1995.0000,2150.0000,1950.0000,1900.0000,1873.0000,2000.0000,1950.0000,1849.0000,1849.0000,2000.0000,2025.0000,2125.0000,2000.0000,2060.0000,2050.0000,1932.0000,1914.0000,1950.0000,1950.0000,1950.0000,2025.0000,2100.0000,2250.0000,2200.0000,2100.0000,2050.0000,2095.0000,2175.0000,2200.0000,2200.0000,2000.0000,2000.0000,1899.0000,2000.0000,2050.0000,2050.0000,2225.0000,2250.0000,2548.0000,2500.0000,2500.0000,2513.0000,2700.0000,2600.0000,2625.0000,2700.0000,2850.0000,2750.0000,2700.0000,2397.0000
eviction_count,31,30,34,33,42,31,32,24,28,45,34,9,42,33,25,41,29,35,23,39,25,37,34,31,27,39,30,36,38,46,31,22,22,30,24,15,28,16,12,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,3,5,0,1,0,3,11,9,13,11,12,9,11,13,12,15,30,20,26,32,27,22,29,58,33,43,14,38,24,25,44,55,45,41,51,25,28,33,31,42,22,25


## **Step 4.2 Covid times (tbc)**